In [20]:
library(gamm4)
library(tidyverse)

In [21]:
df <- read.csv('/home/twalton_umass_edu/Political Polarization Project/dem_20210428.csv')
df$user_id_fac <- as.factor(df$user_id)
schools <- c("s1", "s2", "s3", "s4", "s5", "s6", "s8")
final_coeffs <- data.frame(NULL)

In [22]:
for(k in 4:31){
    for(s in schools){
        school <- df[df$school == s,]
        
        spline_fit_re <- gamm4(
            formula = emot_100_mean ~
            s(period, bs = "cr", k = k, by = log_dem_count) + 
            s(period, bs = "cr", k = k, by = log_rep_count) + 
            user_id_fac, data = school)
        
        cis <- data.frame(confint(spline_fit_re$gam, type = 'confidence'))
        coeffs <- spline_fit_re$gam$coefficients
        
        for(i in 1:k){
            #rep coefficients
            current <- data.frame(coeff = coeffs[[paste0('s(period):log_rep_count.', i)]],
                          lower = cis[paste0('s(period):log_rep_count.', i), ][[1]],
                          upper = cis[paste0('s(period):log_rep_count.', i), ][[2]],
                          period = i, school = s, k = k, aic = AIC(spline_fit_re$mer),
                          coeff_party = 'rep')
            final_coeffs = rbind(current, final_coeffs)
            
            #dem coefficients
            current <- data.frame(coeff = coeffs[[paste0('s(period):log_dem_count.', i)]],
                          lower = cis[paste0('s(period):log_dem_count.', i), ][[1]],
                          upper = cis[paste0('s(period):log_dem_count.', i), ][[2]],
                          period = i, school = s, k = k, aic = AIC(spline_fit_re$mer),
                          coeff_party = 'dem')
            final_coeffs = rbind(current, final_coeffs)
        }
    }
    print(k)
    flush.console()
}

write.csv(final_coeffs, 'dem_gamm_models.csv')

[1] 4
[1] 5
[1] 6
[1] 7
[1] 8
[1] 9
[1] 10
[1] 11
[1] 12
[1] 13
[1] 14
[1] 15
[1] 16
[1] 17
[1] 18
[1] 19
[1] 20
[1] 21
[1] 22
[1] 23
[1] 24
[1] 25
[1] 26
[1] 27
[1] 28
[1] 29
[1] 30
[1] 31


In [8]:
df <- read.csv('rep_gamm_models.csv')

In [14]:
lowest <- df %>%
    group_by(school) %>% slice(which.min(aic))

In [6]:
df <- df[df$aic == min(df$aic), ]